# Coordination Conversion for files

Collect data of Tieliikenneonnettomuudet by years from Avoindata.fi. Data is created by Väylävirasto. The data contains coordinate information in a format ETRSTM35FIN not supported by most data visualization tools. Therefore is a need to convert the coordinates into a format WGS84 that is useful.

This program is made to convert a large number of coordinates in a file to another format.
1. Read file and source coordinations
2. Convert coordinations
3. Create new coordinates and drop source coordinates for file
4. Save converted file to directory

For now, supported coordinate formats are only ETRSTM35FIN and WGS84.

I have made changes to the re-used functions (see below) for the following reason and in the following way:
- I want to process coordinate values from a large number of the file lines (rows) at once
  - in this case it is easiest to process both coordinate values of each row at the same time in the functions
  - it is simple to read and handle values from dataframe
- I have changed the code of the re-used functions in that way:
  - both coordinate values are imported and returned from the functions
---
Re-used parts 'Constants' and some 'Functions' are from free public source code by Olli Lammi.
- File:            coordinates.py
- Author:          Olli Lammi
- Version:         1.0f
- Date:            17.01.2022
- License:         MIT License http://opensource.org/licenses/MIT
-                  Copyright (c) 2012-2015 Olli Lammi olammi@iki.fi

The software is provided "as is", without warranty of any kind, express or implied, including but not limited to the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or otherwise, arising from, out of or in connection with the software or the use or other dealings in the software.

### Define the source and result file parameters

In [1]:
# File path and name
year = 2014
src_file = '../datasets/onnettomuus/tieliikenneonnettomuudet_'+str(year)+'_onnettomuus.csv'
rslt_file = '../datasets/onnettomuus/co/tielonnett_'+str(year)+'_onnett.csv'

# File attributes
src_delimiter = ';'
src_encoding = 'ansi'
rslt_delimiter = ';'
rslt_encoding = 'utf-8'

# Coordinate types (ETRSTM35FIN , WGS84)
src_coord = 'ETRSTM35FIN'
rslt_coord = 'WGS84'

# Coordinate column names in files
src_col_N = 'Y'
src_col_E = 'X'
rslt_col_N = 'position.lat'
rslt_col_E = 'position.lon'

In [2]:
# Used libraries
import pandas as pd
import numpy as np
import math

### Constants

In [3]:
# Ellipsoids
ELLIPSOID = {'WGS84': {'a': 6378137.0, 'b': 6356752.314245, 'f': 1.0 / 298.257223563, 'k0': 0.9996}, \
             'KKJ': {'a': 6378388.0, 'b': 6356911.946128, 'f': 1.0 / 297.0, 'k0': 1.0}, \
             'GRS80': {'a': 6378137.0, 'b': 6356752.314140, 'f': 1.0 / 298.257222101, 'k0': 1.0} \
}

# init precalculated ellipsoid parameters
for key in ELLIPSOID.keys():
    a = ELLIPSOID[key]['a']
    f = ELLIPSOID[key]['f']
    
    n = f / (2.0 - f)
    ELLIPSOID[key]['n'] = n
    ELLIPSOID[key]['A1'] = a / (1.0 + n) * (1.0 + math.pow(n, 2.0) / 4.0 + math.pow(n, 4.0) / 64.0)
    ELLIPSOID[key]['e'] = math.sqrt(2.0 * f - math.pow(f, 2.0))
    ELLIPSOID[key]['h1'] = 1.0/2.0 * n - 2.0/3.0 * math.pow(n, 2.0) + 37.0/96.0 * math.pow(n, 3.0) - 1.0/360.0 * math.pow(n, 4.0)
    ELLIPSOID[key]['h2'] = 1.0/48.0 * math.pow(n, 2.0) + 1.0/15.0 * math.pow(n, 3.0) - 437.0/1440.0 * math.pow(n, 4.0)
    ELLIPSOID[key]['h3'] = 17.0/480.0 * math.pow(n, 3.0) - 37.0/840.0 * math.pow(n, 4.0)
    ELLIPSOID[key]['h4'] = 4397.0/161280.0 * math.pow(n, 4.0)
    ELLIPSOID[key]['h1p'] = 1.0/2.0 * n - 2.0/3.0 * math.pow(n, 2.0) + 5.0/16.0 * math.pow(n, 3.0) + 41.0/180.0 * math.pow(n, 4.0)
    ELLIPSOID[key]['h2p'] = 13.0/48.0 * math.pow(n, 2.0) - 3.0/5.0 * math.pow(n, 3.0) + 557.0/1440.0 * math.pow(n, 4.0)
    ELLIPSOID[key]['h3p'] = 61.0/240.0 * math.pow(n, 3.0) - 103.0/140.0 * math.pow(n, 4.0)
    ELLIPSOID[key]['h4p'] = 49561.0/161280.0 * math.pow(n, 4.0)


# Functions

### Function, main: ETRSTM35FIN_to_WGS84

In [4]:
def ETRSTM35FIN_to_WGS84(ETRSin):
    WGS84lalo = pd.DataFrame(columns = [rslt_col_N, rslt_col_E])
    for i in ETRSin.index:
        E = ETRSin.at[i, src_col_E]
        N = ETRSin.at[i, src_col_N]
        LALO = ETRSTM35FINxy_to_WGS84lalo(E, N)
        WGS84lalo.at[i, rslt_col_N] = float("{:.9f}".format(LALO.get('lat')))
        WGS84lalo.at[i, rslt_col_E] = float("{:.9f}".format(LALO.get('lon')))
    return WGS84lalo

### Function, main: WGS84_to_ETRSTM35FIN

In [5]:
def WGS84_to_ETRSTM35FIN(WGS84in):
    ETRSTM35FINxy = pd.DataFrame(columns = [rslt_col_N, rslt_col_E])
    for i in WGS84in.index:
        lat = WGS84in.at[i, src_col_N]
        lon = WGS84in.at[i, src_col_E]
        XY = WGS84lalo_to_ETRSTM35FINxy(lat, lon)
        ETRSTM35FINxy.at[i, rslt_col_E] = np.round(XY.get('E'), 0)
        ETRSTM35FINxy.at[i, rslt_col_N] = np.round(XY.get('N'), 0)
    return ETRSTM35FINxy

### Function: ETRSTM35FINxy_to_WGS84lalo

Input:
- dataframe with
  - ['N'] is ETRS-TM35FIN Northing
  - ['E'] in ETRS-TM35FIN Easting

Output:
- dataframe with
  - ['lat'] is latitude in degrees (WGS84)
  - ['lon'] is longitude in degrees (WGS84)

In [6]:
def ETRSTM35FINxy_to_WGS84lalo(E, N):  
    lo0 = 27.0
    E0 = 500000.0
  
    return xy_to_lalo(E, N, lo0, E0, ELLIPSOID['WGS84'])

### Function: WGS84lalo_to_ETRSTM35FINxy

Input:
- dataframe with
  - ['lat'] is latitude in degrees (WGS84)
  - ['lon'] is longitude in degrees (WGS84)

Output:
- dataframe with
  - ['N'] is ETRS-TM35FIN Northing
  - ['E'] in ETRS-TM35FIN Easting

In [7]:
def WGS84lalo_to_ETRSTM35FINxy(lat, lon):
  lo0 = 27.0
  E0 = 500000.0
  
  return lalo_to_xy(lat, lon, lo0, E0, ELLIPSOID['WGS84'])

### Function, sub: xy_to_lalo

In [8]:
def xy_to_lalo(x_E, y_N, lo0, E0, ellipsoid):  
    lo0 = math.radians(lo0)

    A1 = ellipsoid['A1']
    k0 = ellipsoid['k0']
    e = ellipsoid['e']
    h1 = ellipsoid['h1']
    h2 = ellipsoid['h2']
    h3 = ellipsoid['h3']
    h4 = ellipsoid['h4']
  
    E = y_N / (A1 * k0)
    nn = (x_E - E0) / (A1 * k0)
  
    E1p = h1 * math.sin(2.0 * E) * math.cosh(2.0 * nn)
    E2p = h2 * math.sin(4.0 * E) * math.cosh(4.0 * nn)
    E3p = h3 * math.sin(6.0 * E) * math.cosh(6.0 * nn)
    E4p = h4 * math.sin(8.0 * E) * math.cosh(8.0 * nn)
    nn1p = h1 * math.cos(2.0 * E) * math.sinh(2.0 * nn)
    nn2p = h2 * math.cos(4.0 * E) * math.sinh(4.0 * nn)
    nn3p = h3 * math.cos(6.0 * E) * math.sinh(6.0 * nn)
    nn4p = h4 * math.cos(8.0 * E) * math.sinh(8.0 * nn)
    Ep = E - E1p - E2p - E3p - E4p
    nnp = nn - nn1p - nn2p - nn3p - nn4p
    be = math.asin(math.sin(Ep) / math.cosh(nnp))
  
    Q = math.asinh(math.tan(be))
    Qp = Q + e * math.atanh(e * math.tanh(Q))
    Qp = Q + e * math.atanh(e * math.tanh(Qp))
    Qp = Q + e * math.atanh(e * math.tanh(Qp))
    Qp = Q + e * math.atanh(e * math.tanh(Qp))
  
    LALO = {}
    LALO['lat'] = float(math.degrees(math.atan(math.sinh(Qp)))) 
    LALO['lon'] = float(math.degrees(lo0 + math.asin(math.tanh(nnp) / math.cos(be))))

    return LALO

### Function, sub: lalo_to_xy

In [9]:
def lalo_to_xy(lat, lon, lo0, E0, ellipsoid):
    lo0 = math.radians(lo0)  
    lat = math.radians(lat)
    lon = math.radians(lon)  
  
    e = ellipsoid['e']
    k0 = ellipsoid['k0']
    h1p = ellipsoid['h1p']
    h2p = ellipsoid['h2p']
    h3p = ellipsoid['h3p']
    h4p = ellipsoid['h4p']
    A1 = ellipsoid['A1']  
    
    Q = math.asinh(math.tan(lat)) - e * math.atanh(e * math.sin(lat))
    be = math.atan(math.sinh(Q))
    nnp = math.atanh(math.cos(be) * math.sin(lon - lo0))
    Ep = math.asin(math.sin(be) * math.cosh(nnp))  
    E1 = h1p * math.sin(2.0 * Ep) * math.cosh(2.0 * nnp)
    E2 = h2p * math.sin(4.0 * Ep) * math.cosh(4.0 * nnp)
    E3 = h3p * math.sin(6.0 * Ep) * math.cosh(6.0 * nnp)
    E4 = h4p * math.sin(8.0 * Ep) * math.cosh(8.0 * nnp)
    nn1 = h1p * math.cos(2.0 * Ep) * math.sinh(2.0 * nnp)
    nn2 = h2p * math.cos(4.0 * Ep) * math.sinh(4.0 * nnp)
    nn3 = h3p * math.cos(6.0 * Ep) * math.sinh(6.0 * nnp)
    nn4 = h4p * math.cos(8.0 * Ep) * math.sinh(8.0 * nnp)
    E = Ep + E1 + E2 + E3 + E4
    nn = nnp + nn1 + nn2 + nn3 + nn4
  
    XY = {}
    XY['N'] = A1 * E * k0
    XY['E'] = A1 * nn * k0 + E0

    return XY

## Conversion

#### Download source file

In [10]:
df = pd.read_csv(src_file, delimiter=src_delimiter, encoding=src_encoding)

C:\Users\Pasi\AppData\Local\Temp\ipykernel_9236\842088147.py:1: DtypeWarning: Columns (68) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(src_file, delimiter=src_delimiter, encoding=src_encoding)


### Select conversation type and needed operations

In [11]:
if src_coord == 'ETRSTM35FIN' and rslt_coord == 'WGS84':
    # Slice needed columns from source file
    ETRSin = pd.DataFrame(df[[src_col_N,src_col_E]])
    # ETRSTM35FIN (x, y) --> WGS84 (lat, lon)
    WGS84lalo = ETRSTM35FIN_to_WGS84(ETRSin)
    # Concate converted file and source file (converted columns to the end of source file columns)
    frames = [df, WGS84lalo]
    result = pd.concat(frames, axis=1)
elif src_coord == 'WGS84' and rslt_coord == 'ETRSTM35FIN':
    # Slice needed columns from source file
    WGS84in = pd.DataFrame(df[[src_col_N,src_col_E]])
    # WGS84 (lat, lon) --> ETRSTM35FIN (x, y)
    ETRSTM35FINxy = WGS84_to_ETRSTM35FIN(WGS84in)
    # Concate converted file and source file (converted columns to the end of source file columns)
    frames = [df, ETRSTM35FINxy]
    result = pd.concat(frames, axis=1)

## Finished

#### Drop old coordination columns

In [12]:
result_file = result.drop([src_col_N, src_col_E], axis=1)

#### Create new csv file with converted values

In [13]:
result_file.to_csv(rslt_file, sep=rslt_delimiter, encoding=rslt_encoding, index=False)